In [1]:
import requests

api_key = "20211595:29e2994c-2337-48d4-a828-4a8f424edfd0"
endpoint = "https://iaapi.novaims.unl.pt/v1/completions"
def llama_completion(prompt, max_tokens=1024):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "Gemma3:12b",
        "prompt": prompt,
        "max_tokens": max_tokens
    }
    response = requests.post(endpoint, headers=headers, json=data)
    return response.json()

def main():
    prompt = "Conta uma anedota sobre programadores"
    response = llama_completion(prompt)
    print(response["choices"][0]["text"])

if __name__ == "__main__":
    main()

Com prazer! Aqui vai uma anedota clássica sobre programadores:

Um programador está sendo entrevistado para uma vaga. O entrevistador decide testá-lo com uma pergunta simples:

"Como você resolveria o problema de uma geladeira que não está funcionando?"

O programador, sem hesitar, responde:

"Primeiro, eu verificaria se ela está ligada."

O entrevistador, um pouco decepcionado, pergunta: "E se ela estiver ligada?"

O programador responde: "Então eu verificaria se a tomada está funcionando."

"E se a tomada estiver funcionando?"

"Eu checaria se o fio está conectado."

A conversa continua nesse ritmo, com o programador oferecendo soluções cada vez mais básicas e lógicas. O entrevistador, já impaciente, explode:

"Para com isso! Você está me dando respostas óbvias! Você não pode resolver problemas complexos?"

E o programador, com a maior naturalidade do mundo, responde:

"Eu estou resolvendo. Estou resolvendo o problema de uma geladeira que não está funcionando, um passo de cada vez."


**ESSENCIAL**

In [2]:
import os
from openai import AzureOpenAI
import api

endpoint = "https://ai-bcds.openai.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"

subscription_key = api.local_settings.API_KEY
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)


8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ
https://ai-bcds.openai.azure.com/


**CHATBOT BASE sem ler documentos**

In [3]:
# === CHAT FUNCTION ===
def ask_financial_bot(user_input: str) -> str:
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful and professional financial assistant. "
                "Always give clear, trustworthy financial advice, and explain things in a beginner-friendly way."
            )
        },
        {
            "role": "user",
            "content": user_input
        }
        
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=1024,
        temperature=0.7,
        top_p=1.0
    )

    return response.choices[0].message.content

# === EXAMPLE USE ===
if __name__ == "__main__":
    question = input("Ask your financial assistant: ")
    answer = ask_financial_bot(question)
    print("\n💬 Answer:", answer)



💬 Answer: Choosing the best investments depends on various factors, including your financial goals, risk tolerance, investment horizon, and current market conditions. Here are some common investment options along with their characteristics:

1. **Stocks**: Investing in individual companies can offer high returns, but it also comes with higher risk. Stocks are best for those who are willing to ride out market fluctuations.

2. **Bonds**: Bonds are generally considered safer than stocks and provide fixed interest payments over time. They’re suitable for conservative investors looking for steady income.

3. **Mutual Funds**: These are professionally managed investment funds that pool money from many investors to buy a diversified portfolio of stocks or bonds. They are good for beginners looking for diversification.

4. **Exchange-Traded Funds (ETFs)**: Similar to mutual funds, ETFs trade on stock exchanges and typically track an index. They offer diversification and lower fees than mutua

**CHATBOT a ler 1 documento**

In [11]:
import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join(page.get_text() for page in doc)

pdf_text = extract_text_from_pdf("docs/Documents for training and evaluation-20250507/Questions_Answers_Censored.pdf")

def ask_with_pdf_context(user_input):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a financial assistant. Use the following document as context:\n\n"
                + pdf_text[:5000]  # Keep it short to avoid hitting token limits
            )
        },
        {"role": "user", "content": user_input}
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


In [12]:
# === EXAMPLE USE ===
if __name__ == "__main__":
    question = input("Ask your financial assistant: ")
    answer = ask_with_pdf_context(question)
    print("\n💬 Answer:", answer)



💬 Answer: O Fidelidade Savings é indicado para diferentes perfis de risco:

1. **Clientes conservadores**: Opção Seguro, que oferece capital e rendimento garantido.
2. **Clientes com risco moderado**: Opção Proteção, que garante 90% do capital investido no vencimento e tem exposição ao Índice Multiativos.
3. **Clientes propensos ao risco**: Opção Dinâmico, que possui 100% de exposição ao Índice Multiativos.
4. **Clientes com preocupações Ambientais, Sociais e de Governança (ESG)**: Opção ESG, que investe no índice Multiativos ESG, apresentando risco mas com foco em critérios ESG.


**CHATBOT a ler vários documentos**

In [13]:
import fitz

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join(page.get_text() for page in doc)

In [14]:
import os

pdf_dir = "docs/Documents for training and evaluation-20250507"
all_texts = []

for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        path = os.path.join(pdf_dir, pdf_file)
        text = extract_text_from_pdf(path)
        all_texts.append(text)


In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.create_documents(all_texts)
texts = [chunk.page_content for chunk in chunks]


In [16]:
from openai import AzureOpenAI
import numpy as np

embedding_client = AzureOpenAI(
    api_key="8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
    api_version="2024-02-01",
    azure_endpoint="https://ai-bcds.openai.azure.com/"
)

def get_embedding(text):
    response = embedding_client.embeddings.create(
        input=[text],
        model="text-embedding-3-small"  # Use the deployment name you set up
    )
    return np.array(response.data[0].embedding).astype('float32')

In [17]:
#%pip install faiss-cpu
import faiss

In [18]:
embeddings = np.array([get_embedding(text) for text in texts])

# Check the shape of embeddings (Should be (N, D) where N is number of chunks and D is embedding dimension)
print(embeddings.shape)

# Use IndexFlatL2 (no training needed)
d = 1536  # Dimensionality of the embeddings (1536 for text-embedding-3-small)
index = faiss.IndexFlatL2(d)  # This index does not require training

# Add embeddings to the index
index.add(embeddings)


(18, 1536)


In [19]:
def get_top_k_chunks(query, k=3):
    query_vec = np.array([get_embedding(query)]).astype('float32')
    D, I = index.search(query_vec, k)  # Get top-k similar chunks
    return [texts[i] for i in I[0]]  # Return the corresponding text chunks

In [20]:
# Example usage
query = "What are the latest trends in financial advising?"
top_chunks = get_top_k_chunks(query)
print(top_chunks)

['1) FIDELIDADE SAVINGS \nInformações Gerais sobre os Produtos \n• \nO Fidelidade Savings é mais indicado para que perfil de risco? \nR: o Fidelidade Savings é adequado para clientes conservadores, através da \nsua opção Seguro, com capital e rendimento garantido. Para clientes que \ngostam moderadamente do risco, através da opção Proteção com garantia de \n90% do capital investido no vencimento e exposição ao Índice Multiativos. Para \nclientes propensos ao risco através da opção Dinâmico com 100% de', "clientes propensos ao risco através da opção Dinâmico com 100% de \nexposição ao Índice Multiativos. Também está disponível a opção ESG, que \ninveste no índice Multiativos ESG, com risco mas ESG, para clientes com \npreocupações Ambientais, Sociais e de Governança. \n• \nFiz uma poupança Kid's para os meus 3 filhos, como garanto que cada um tem \ndireito ao que lá tem? \nR: Deverá alterar a cláusula beneficiária, solicitando a um agente Fidelidade", 'Rentabilidade e Performance \n• \n

Copilot

In [21]:
# Cell 1: Imports and PDF extraction
import os
import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join(page.get_text() for page in doc)

In [28]:
# Cell 2: Read all PDFs and store their texts with filenames
pdf_dir = "docs/Documents for training and evaluation-20250507/PPR Evoluir_Documents/PPR Evoluir - Public Information"
file_texts = {}

for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        path = os.path.join(pdf_dir, pdf_file)
        text = extract_text_from_pdf(path)
        file_texts[pdf_file] = text

In [29]:
# Cell 3: Split each file's text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
file_chunks = {}

for fname, text in file_texts.items():
    docs = splitter.create_documents([text])
    file_chunks[fname] = [chunk.page_content for chunk in docs]

In [30]:
# Cell 4: Get embeddings for all chunks, keeping track of which file they came from
from openai import AzureOpenAI
import numpy as np

embedding_client = AzureOpenAI(
    api_key="8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",  # Replace with your key
    api_version="2024-02-01",
    azure_endpoint="https://ai-bcds.openai.azure.com/"
)

def get_embedding(text):
    response = embedding_client.embeddings.create(
        input=[text],
        model="text-embedding-3-small"
    )
    return np.array(response.data[0].embedding).astype('float32')

all_chunks = []
chunk_file_map = []
for fname, chunks in file_chunks.items():
    for chunk in chunks:
        all_chunks.append(chunk)
        chunk_file_map.append(fname)

embeddings = np.array([get_embedding(text) for text in all_chunks])

In [31]:
# Cell 5: Build FAISS index
import faiss

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)

In [32]:
# Cell 6: Search for top-k chunks per file and combine info
def search_across_files(query, k=2):
    query_vec = np.array([get_embedding(query)]).astype('float32')
    D, I = index.search(query_vec, k * len(file_chunks))  # Get more to filter per file

    # Collect top-k per file
    per_file_results = {fname: [] for fname in file_chunks}
    for idx in I[0]:
        fname = chunk_file_map[idx]
        if len(per_file_results[fname]) < k:
            per_file_results[fname].append(all_chunks[idx])

    # Combine info from all files
    combined = []
    for fname, chunks in per_file_results.items():
        if chunks:
            combined.append(f"From {fname}:\n" + "\n".join(chunks))
    return "\n\n".join(combined)

In [34]:
# Cell 7: Example usage
query = "O que é o PPR Evoluir?"
answer = search_across_files(query, k=2)
print(answer)

From IPC_PPR Evoluir ParticularesJAN2025.pdf:
4. GARANTIAS
O seguro PPR EVOLUIR é um Plano de Poupança-Reforma – PPR constituído sob a forma de Fundos Autónomos de uma modalidade de seguro do «Ramo 
Vida», que apenas pode ser subscrito em simultâneo nas 2 Componentes de investimento que o Segurador disponibiliza para o efeito, nas proporções 
correspondentes à idade da Pessoa Segura no início ou no aniversário anterior da apólice, com um máximo de 60%.
O contrato garante ao beneficiário:
1/7
Seguro PPR Evoluir - janeiro 2025 - I763700
INFORMAÇÕES PRÉ-CONTRATUAIS 
SEGURO PPR EVOLUIR
POUPANÇA E INVESTIMENTO
Fidelidade - Companhia de Seguros, S.A. - NIPC e Matrícula 500 918 880, na CRC Lisboa • Sede: Largo do Calhariz, 30 • 1249-001 Lisboa - Portugal • Capital Social EUR 509.263.524
Apoio ao Cliente: Dias úteis das 9h às 20h • T. 217 94 87 01 Chamada para a rede fixa nacional • E. apoiocliente@fidelidade.pt • www.fidelidade.pt
1. SEGURADOR

From CG-PPR Evoluir (Rede)JAN2025.pdf:
ANEXO
SEG

In [11]:
import numpy as np
from openai import AzureOpenAI


# Cell 4: Get embeddings for all chunks, keeping track of which file they came from
from openai import AzureOpenAI
import numpy as np

embedding_client = AzureOpenAI(
    api_key="8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",  # Replace with your key
    api_version="2024-02-01",
    azure_endpoint="https://ai-bcds.openai.azure.com/"
)

# Cell 1: Imports and PDF extraction
import os
import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join(page.get_text() for page in doc)

# Cell 2: Read all PDFs and store their texts with filenames
pdf_dir = "docs/Documents for training and evaluation-20250507/PPR Evoluir_Documents/PPR Evoluir - Public Information"
file_texts = {}

import os

for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        path = os.path.join(pdf_dir, pdf_file)
        text = extract_text_from_pdf(path)
        file_texts[pdf_file] = text

def get_embedding(text):
    response = embedding_client.embeddings.create(
        input=[text],
        model="text-embedding-3-small"
    )
    return np.array(response.data[0].embedding).astype('float32')


# Cell 3: Split each file's text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
file_chunks = {}

for fname, text in file_texts.items():
    docs = splitter.create_documents([text])
    file_chunks[fname] = [chunk.page_content for chunk in docs]
    
all_chunks = []
chunk_file_map = []
for fname, chunks in file_chunks.items():
    for chunk in chunks:
        all_chunks.append(chunk)
        chunk_file_map.append(fname)

embeddings = np.array([get_embedding(text) for text in all_chunks])

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)

# Cell 6: Retrieve top-N most relevant chunks globally, then synthesize an answer
def get_top_n_chunks(query, n=5):
    query_vec = np.array([get_embedding(query)]).astype('float32')
    D, I = index.search(query_vec, n)
    results = []
    for idx in I[0]:
        fname = chunk_file_map[idx]
        chunk = all_chunks[idx]
        results.append((fname, chunk))
    return results

def synthesize_answer(query, n=5):
    # Get top-N relevant chunks from all files
    top_chunks = get_top_n_chunks(query, n)
    # Build context string
    context = "\n\n".join([f"From {fname}:\n{chunk}" for fname, chunk in top_chunks])
    # Compose prompt for LLM
    prompt = (
        f"Use the following information extracted from several documents to answer the user's question.\n\n"
        f"Context:\n{context}\n\n"
        f"User question: {query}\n\n"
        f"Answer in a clear, concise, and professional way:"
    )
    # Call your LLM (using your AzureOpenAI client)
    response = embedding_client.chat.completions.create(
        model="gpt-4o-mini",  # or your deployment name
        messages=[
            {"role": "system", "content": "You are a helpful financial assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=512,
        temperature=0.2
    )
    return response.choices[0].message.content

# Example usage
query = "FUNDO AUTÓNOMO DE INVESTIMENTO"
answer = synthesize_answer(query, n=5)
print(answer)

The "Fundo Autónomo de Investimento" refers to a specific investment fund structure utilized within the context of certain insurance contracts, particularly those related to pension plans (PPR). The assets representing the technical provisions of these contracts are invested in this autonomous fund, as outlined in the respective special conditions.

Key features of the Fundo Autónomo include:

1. **Investment Composition**: The fund's assets are primarily expressed in euros, with a maximum of 20% allowed in other currencies. The investment portfolio is diversified across various instruments, with specific limits on the allocation to different asset classes.

2. **Investment Components**: Each investment component corresponds to a distinct Fundo Autónomo, which has unique characteristics, including asset composition, investment policies, and management fees.

3. **Recomposition of Investment**: The contract allows for automatic recompositions between the active investment component and 